In [ ]:
from vtkmodules.vtkIOXML import vtkXMLPolyDataReader
from vtkmodules.vtkCommonDataModel import vtkPolyData
from vtkmodules.vtkFiltersModeling import vtkRuledSurfaceFilter

filename = "~/Temp/"

reader = vtkXMLPolyDataReader()
reader.SetFileName(filename)
reader.Update()

ruledSurfaceFilter = vtkRuledSurfaceFilter()
ruledSurfaceFilter.SetInputData(reader.GetOutput())
ruledSurfaceFilter.SetResolution(21, 21)
ruledSurfaceFilter.SetRuledModeToResample()

In [1]:
import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkCommonCore import vtkPoints
from vtkmodules.vtkCommonDataModel import (
    vtkCellArray,
    vtkLine,
    vtkPolyData
)
from vtkmodules.vtkFiltersModeling import vtkRuledSurfaceFilter
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)


def main():
    colors = vtkNamedColors()

    # Create a rendering window and renderer
    ren = vtkRenderer()
    renWin = vtkRenderWindow()
    renWin.AddRenderer(ren)

    # Create a renderwindowinteractor
    iren = vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)

    # Create the points fot the lines.
    points = vtkPoints()
    points.InsertPoint(0, 0, 0, 1)
    points.InsertPoint(1, 1, 0, 0)
    points.InsertPoint(2, 0, 1, 0)
    points.InsertPoint(3, 1, 1, 1)

    # Create line1
    line1 = vtkLine()
    line1.GetPointIds().SetId(0, 0)
    line1.GetPointIds().SetId(1, 1)

    # Create line2
    line2 = vtkLine()
    line2.GetPointIds().SetId(0, 2)
    line2.GetPointIds().SetId(1, 3)

    # Create a cellArray containing the lines
    lines = vtkCellArray()
    lines.InsertNextCell(line1)
    lines.InsertNextCell(line2)

    # Create the vtkPolyData to contain the points and cellArray with the lines
    polydata = vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetLines(lines)

    # Create the ruledSurfaceFilter from the polydata containing the lines
    ruledSurfaceFilter = vtkRuledSurfaceFilter()
    ruledSurfaceFilter.SetInputData(polydata)
    ruledSurfaceFilter.SetResolution(21, 21)
    ruledSurfaceFilter.SetRuledModeToResample()

    # Create the mapper with the ruledSurfaceFilter as input
    mapper = vtkPolyDataMapper()
    mapper.SetInputConnection(ruledSurfaceFilter.GetOutputPort())

    # Create the actor with the mapper
    actor = vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(colors.GetColor3d("Banana"))
    actor.GetProperty().SetSpecular(0.6)
    actor.GetProperty().SetSpecularPower(30)

    # Add the actor to the display
    ren.AddActor(actor)
    ren.SetBackground(colors.GetColor3d("SteelBlue"))
    renWin.SetWindowName('RuledSurfaceFilter')

    # Enable user interface interactor
    iren.Initialize()
    renWin.Render()
    # ren.GetActiveCamera().SetPosition(3.7, -0.5, -0.5)
    # ren.GetActiveCamera().SetFocalPoint(0.5, 0.5, 0.5)
    # ren.GetActiveCamera().SetViewUp(-0.3, 0.1, -1.0)
    ren.GetActiveCamera().Azimuth(60)
    ren.GetActiveCamera().Elevation(60)
    ren.ResetCamera()

    renWin.Render()
    iren.Start()

main()

: 

# Feature flow field

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from maxpy.algorithms import *
from maxpy.utils import save_as_vti

size = 32
linx = np.linspace(-4, 4, size)
liny = np.linspace(-4, 4, size)
linz = np.linspace(-3, 3, size)

origin = [linx[0], liny[0], linz[0]]
spacing = [linx[1] - linx[0], liny[1] - liny[0], linz[1] - linz[0]]

x, y, z = np.meshgrid(linx, liny, linz, indexing="ij")

field = np.array([
    (y -2) ** 2 - 1 + z**2, 
    -x, 
    np.ones(x.shape) * 0.1
]) 

dirDiv = get_directional_derivative(field, linx, liny, linz)

Jac = get_jacobian(dirDiv, linx, liny, linz)#.transpose(2,3,4,0,1).reshape(-1,3,3)
print(dirDiv.shape)

#dirDiv = dirDiv.transpose(1,2,3,0).reshape(-1,3)

f = np.zeros(field.shape)

for i in range(size):
    for j in range(size):
        for k in range(size):
            f[0,i,j,k] = np.cross(Jac[1,:, i, j, k], Jac[2, :,i, j, k])@field[:,i,j,k]
            f[1,i,j,k] = np.cross(Jac[2,:, i, j, k], Jac[0, :,i, j, k])@field[:,i,j,k]
            f[2,i,j,k] = np.cross(Jac[0,:, i, j, k], Jac[1, :,i, j, k])@field[:,i,j,k]


save_as_vti(f.reshape(3,*x.shape), None, savepath="~/Temp/", name="feature-flow-field.vti")
save_as_vti(field, dirDiv, savepath="~/Temp/", name="feature-flow-field-field.vti")

(3, 32, 32, 32)
